# Import packages and data 

In [1]:
import numpy as np 
import pandas as pd 
import pylab as pl 

In [16]:
net = pd.read_parquet('../local_data/tables/links_ifis_w_downstream.gz')
net.set_index('LINKNO', inplace = True)
usgs = pd.read_csv('../local_data/tables/usgs_temporal_linked_w_trimmed.csv')
ifc = pd.read_csv('../local_data/tables/ifc_linked_w_trimmed.csv')

# Getting the links upstream and downstream of each gauge 

In [173]:
def get_lids_downstream(lid, net):
    '''Function that gets the list of downstream segments given a linkid'''
    #Define list of lids 
    lids_down = []
    while lid > 0:
        lids_down.append(lid)
        lid = net.loc[lid, 'ds']
    return lids_down


def get_lids_upstream(lid, net, max_length = 1500):
    '''Function to get the upstream segments of a given lid along the main channel.
    Stop when the accumulated length is above the max_length'''
    #Define local variables 
    length = net.Length.values
    area = net.DSContArea.values
    lids = net.index.values
    ds = net.ds.values
    tot_lenght = length[np.where(lid == lid)[0]]
    lids_up = []
    flag = True
    while flag:
        up = np.where(ds == lid)[0]
        if up.size > 0:
            pos = up[np.argmax(area[up])]
            lid = lids[pos]
            tot_lenght += length[pos]
            lids_up.append(lid)
        else:
            flag = False            
        if tot_lenght > max_length:
            flag = False
    return lids_up

In [168]:
lids_down = []
for lid in usgs.LINKNO.values:
    lids_down.extend(get_lids_downstream(lid = lid))
for lid in ifc.LINKNO.values:
    lids_down.extend(get_lids_downstream(lid = lid))

In [169]:
#lids_up = []
for lid in usgs.LINKNO.values:
    lids_down.extend(get_lids_upstream(lid = lid))
for lid in ifc.LINKNO.values:
    lids_down.extend(get_lids_upstream(lid = lid))

In [170]:
lids_down = np.unique(np.array(lids_down))

In [171]:
len(lids_down)

23326

In [172]:
f = open('../local_data/tables/lids_up_and_down_gauges.csv','w')
f.write('lid\n')
for lid in lids_down:
    f.write('%d\n' % lid)
f.close()